In [1]:
#!c1.8
%pip list

Package                       Version
----------------------------- ------------------
absl-py                       1.4.0
aiohttp                       3.8.1
aiosignal                     1.2.0
antlr4-python3-runtime        4.8
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.2.1
astunparse                    1.6.3
async-timeout                 4.0.2
atpublic                      2.3
attrs                         23.1.0
audioread                     2.1.9
autograd                      1.3
aws-sam-translator            1.42.0
aws-xray-sdk                  2.9.0
backcall                      0.2.0
bcrypt                        3.2.0
beautifulsoup4                4.12.2
bleach                        6.0.0
blis                          0.7.5
boto                          2.49.0
boto3                         1.12.31
botocore                      1.15.49
Bottleneck                    1.3.2
cachetool

In [2]:
#!c1.8
import zipfile
from tqdm import tqdm

fname = 'plates.zip'
path = './'

with zipfile.ZipFile(fname, 'r') as zf:
    for entry in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(entry, path)
        except zipfile.error as e:
            pass

Extracting : 100%|██████████| 3130/3130 [00:04<00:00, 734.56it/s]


In [18]:
#!c1.8
from tflite_model_maker.config import ExportFormat 
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

import numpy as np
import os

In [19]:
#!c1.8

label_map = ['0']

train_images_dir = './plates/train'
train_annotations_dir = './plates/train'
val_images_dir = './plates/valid'
val_annotations_dir = './plates/valid'
test_images_dir = './plates/test'
test_annotations_dir = './plates/test'

In [20]:
#!c1.8

train_data = object_detector.DataLoader.from_pascal_voc(
    images_dir=train_images_dir,
    annotations_dir=train_annotations_dir,
    label_map=label_map, 
    ignore_difficult_instances=True)
validation_data = object_detector.DataLoader.from_pascal_voc(
    images_dir=val_images_dir,
    annotations_dir=val_annotations_dir, 
    label_map=label_map, 
    ignore_difficult_instances=True)
test_data = object_detector.DataLoader.from_pascal_voc(
    images_dir=test_images_dir, 
    annotations_dir=test_annotations_dir, 
    label_map=label_map, 
    ignore_difficult_instances=True)
 
    
print('train count:', len(train_data))
print('validation count:', len(validation_data))
print('test count:', len(test_data))

train count: 687
validation count: 60
test count: 34


In [21]:
#!c1.8
spec = object_detector.EfficientDetLite0Spec()

In [22]:
#!c1.8
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=120, 
                               batch_size=10, 
                               train_whole_model=True)

2023-06-13 19:03:48.601361: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-06-13 19:05:28.698868: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-06-13 19:06:47.618107: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-06-13 19:08:05.360588: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2023-06-13 19:09:23.088616: W tensorflow/core/framework/dataset.cc:7

68/68 [==============================] - 76s 1s/step - det_loss: 0.1722 - cls_loss: 0.1151 - box_loss: 0.0011 - reg_l2_loss: 0.0671 - loss: 0.2393 - learning_rate: 0.0025 - gradient_norm: 2.1083 - val_det_loss: 0.2799 - val_cls_loss: 0.1897 - val_box_loss: 0.0018 - val_reg_l2_loss: 0.0671 - val_loss: 0.3470
Epoch 85/120
68/68 [==============================] - 79s 1s/step - det_loss: 0.1624 - cls_loss: 0.1104 - box_loss: 0.0010 - reg_l2_loss: 0.0671 - loss: 0.2295 - learning_rate: 0.0024 - gradient_norm: 2.0139 - val_det_loss: 0.2619 - val_cls_loss: 0.1742 - val_box_loss: 0.0018 - val_reg_l2_loss: 0.0671 - val_loss: 0.3290
Epoch 86/120
68/68 [==============================] - 76s 1s/step - det_loss: 0.1670 - cls_loss: 0.1140 - box_loss: 0.0011 - reg_l2_loss: 0.0671 - loss: 0.2341 - learning_rate: 0.0023 - gradient_norm: 2.1517 - val_det_loss: 0.2605 - val_cls_loss: 0.1639 - val_box_loss: 0.0019 - val_reg_l2_loss: 0.0671 - val_loss: 0.3276
Epoch 87/120
68/68 [===========================

In [26]:
#!c1.8
model.evaluate(test_data)

NameError: name 'model' is not defined

In [24]:
#!c1.8
TFLITE_FILENAME = 'efficientdet-lite-platev3.tflite'
LABELS_FILENAME = 'plate-labels.txt'

In [25]:
#!c1.8

model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

2023-06-13 21:39:10.690192: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2023-06-13 21:39:30.378563: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2023-06-13 21:39:38.177385: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-06-13 21:39:38.177432: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0


Estimated count of arithmetic ops: 1.752 G  ops, equivalently 0.876 G  MACs
Estimated count of arithmetic ops: 1.752 G  ops, equivalently 0.876 G  MACs
